### Visualizing Behavior

#### J. Carpenter 2020

In [4]:
# import libraries
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

In [7]:
# Load dataset
fileLoc = "D:\\Data\\Dataset\\24116.mat"
matfile = sio.loadmat(fileLoc) # load .mat file

C:\Users\17145\Anaconda3\lib\site-packages\scipy\io\matlab\mio.py:224: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [40]:
SpikeTrain = matfile["SpikeTrain"][0][11]
np.size(SpikeTrain)

0

### Define functions:

In [79]:
def makeDF(matfile, sessNum, unitNum):
    # make dataframe to pass through plotly for one session
    
    pos = matfile["pos"][0] # time & LEDs
    time = pos[sessNum][:,0]
    led1A = pos[sessNum][:,1]; led1B = pos[sessNum][:,2];
    led2A = pos[sessNum][:,3]; led2B = pos[sessNum][:,4];
      
    SpikeTrain = matfile["SpikeTrain"][0][sessNum][0]    
    if np.size(SpikeTrain) > 0: 
        SpikeTrainUnit = SpikeTrain[unitNum][0]
        d = {'time': time, 'led1A': led1A , 'led1B': led1B, 'led2A': led2A, 'led2B': led2B, 'spikes:': SpikeTrainUnit}
    else:
        d = {'time': time, 'led1A': led1A , 'led1B': led1B, 'led2A': led2A, 'led2B': led2B}
    df = pd.DataFrame(data=d)
    
    return df

In [81]:
def makeAnimation(df):
    px.scatter(df, x="led1A", y="led1B", animation_frame="time")

In [ ]:
makeAnimation(df)

In [447]:
def getUnitData(matfile, sessNum, unitNum):
    
    """
    The following data will be loaded (via scipy.io):
    
        'SpikeTrain':    Sx1 array, where S is the number of sessions for the animal. 
                         Each session (S) contains a Nx1 vector, where N is the number of 
                         neurons (units) recorded in a given session.
                      
        'pos':           Sx1 array containing information about time and the positions of 2 LEDs.
        
        'UniqueID':      Sx1 array, where each array contains an Nx1 vector. Each element is a 
                         string value, containing the unique cell ID for the given unit.
        
        'trialType':    Sx1 array, containing a string value of session type (FM or OF, for
                        'Foster Maze' or 'Open Field', respectively. 'TS' might also be present,
                         which stands for 'Training Session'.
                         
    Outputs:
    
        'time':         tx1 array, where t is the number of timestamps for the session.
    """
    
        
    # pull out relevant arrays
    SpikeTrain = matfile["SpikeTrain"][0]
    pos = matfile["pos"][0] # time & LEDs
    UniqueID = matfile["UniqueID"][0]
    trialType = matfile["trialType"][0]
    
    
    # trial type, spike train, unique ID #
    spikesNow = SpikeTrain[sessNum][0][unitNum][0]
    trialNow = str(trialType[sessNum][0])
    idNow = str(UniqueID[sessNum][0][unitNum][0][0])
    info = [idNow, sessNum, trialNow]
    
    # get window maximums and minumums
    minX = float(matfile["sessInfo"][0][sessNum][0]["window_min_x"][0][0][0][0])
    maxX = float(matfile["sessInfo"][0][sessNum][0]["window_max_x"][0][0][0][0])
    minY = float(matfile["sessInfo"][0][sessNum][0]["window_min_y"][0][0][0][0])
    maxY = float(matfile["sessInfo"][0][sessNum][0]["window_max_y"][0][0][0][0])
    window = np.array([minX, maxX, minY, maxY])
    
    return window, spikesNow, info

In [ ]:
def getTimePt(matfile, sessNum, timePoint):
    
    # parse position vector
    pos = matfile["pos"][0]
    time = pos[sessNum][:,0][timePoint];
    led1A = pos[sessNum][:,1][timePoint]; led1B = pos[sessNum][:,2][timePoint];
    led2A = pos[sessNum][:,3][timePoint]; led2B = pos[sessNum][:,4][timePoint];
    led = np.array([time, led1A, led1B, led2A, led2B])
    
    return time, led

In [ ]:
def plotBehavior(led, window, info):
    """
    plots behavior for a single timepoint.
    """
    
    # parse window vector
    minX = window[0]; maxX = window[1]; minY = window[2]; maxY = window[3]
    
    # parse led vector
    time = led[0]
    L1A = led[1]; L1B = led[2]; L2A = led[3]; L2B = led[4]
    
    # make title
    plot_title = 'uid:' + str(info[0]) + ' / sess:' + str(info[1]) + ' / type:' + str(info[2]) + '/ time:' + str(time)
    
    # plot LEDs at timepoint
    fig, ax = plt.subplots()
    
    ax.hlines(y = minY, xmin = minX, xmax = maxX) #minY
    ax.hlines(y = maxY, xmin = minX, xmax = maxX) # maxY
    ax.vlines(x = minX, ymin = minY, ymax = maxY) # minX
    ax.vlines(x = maxX, ymin = minY, ymax = maxY) # maxX
    
    ax.scatter(L1A, L1B, s=5, c='red')
    ax.scatter(L2A, L2B, s=5, c='blue')
    
    ax.set_xlabel("x");
    ax.set_ylabel("y");
    ax.set_title(plot_title);
    
    plt.show()
    return

### Make animation:

In [ ]:
# pull out data for current session > neuron

# for sessNum in range(0,len(pos)-1): # through sessions
for sessNum in range(0,1):
    SpikeTrain = matfile["SpikeTrain"][0][sessNum][0]
    for unitNum in range(0,len(SpikeTrain[sessNum][0])): #through units
        window, spikesNow, info = getUnitData(matfile, sessNum, unitNum) 
#         for t in range(0,len(time)-1): # through timestamps
        for timePoint in range(0,10): 
            time, led = getTimePt(matfile, sessNum, timePoint)
            plotBehavior(led, window, info)